as of 1/1/2024 Notes: this did not work because googlegenerativeai does not work with openaifunctionsagent

In [2]:
import os
import pandas as pd
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool, PythonAstREPLTool
from langchain.agents import load_tools, initialize_agent, AgentType
from getpass import getpass
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage
from langchain.agents import AgentExecutor

In [4]:
from dotenv import load_dotenv
load_dotenv('../../.env')
google_api_key = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
#Import LLM and Embedding Models
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.embeddings import GooglePalmEmbeddings

In [31]:
llm = GoogleGenerativeAI(model="models/text-bison-001",temperature=0, google_api_key=google_api_key)
embeddings=GooglePalmEmbeddings(model_name="models/embedding-gecko-001",google_api_key=google_api_key)
OpenAIllm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0,model_name="gpt-3.5-turbo-1106")
gchatllm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0, google_api_key=google_api_key, convert_system_message_to_human=True)

In [5]:
from langchain.callbacks.manager import tracing_v2_enabled
from langsmith import Client
client = Client()

LANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT="palm-ragfusion-shadow-99"  # if not specified, defaults to "default"
LANGCHAIN_ENDPOINT=os.getenv("LANGCHAIN_ENDPOINT")
OPENAI_API_KEY=OPENAI_API_KEY

In [ ]:
dset = pd.read_csv('../../ado_backlog_clean.csv')

In [16]:
dset.head(3)

,Work Item Id,Work Item Type,Project Name,Iteration Path,Assigned To,Area Path,Date,Created Date,Activated Date,Resolved Date,Closed Date,Cycle Time Resolved,Changed Date,Iteration Start Date,Iteration End Date,State,State Category,Is Current
0,1013432,User Story,MP-8465878480,AR-8465801115,NaN,NaN,2023-07-18 00:00:00,2020-08-04 00:00:00,2020-08-05 00:00:00,2020-08-05 00:00:00,2020-08-05 00:00:00,1,2023-03-31 00:00:00,2020-08-02 00:00:00,2020-08-14 00:00:00,Closed,Completed,True
1,1027305,Bug,MP-8465878480,AR-8465801115,AM-6710111097,AR-8465878480,2023-07-18 00:00:00,2020-08-07 00:00:00,2020-08-10 00:00:00,2020-08-10 00:00:00,2020-08-11 00:00:00,2,2021-08-20 00:00:00,2020-08-02 00:00:00,2020-08-14 00:00:00,Closed,Completed,True
2,1033357,Bug,MP-8465878480,AR-8465801115,AM-6710111097,AR-8465878480,2023-07-18 00:00:00,2020-08-10 00:00:00,2020-08-10 00:00:00,2020-08-10 00:00:00,2020-08-12 00:00:00,3,2021-08-20 00:00:00,2020-08-02 00:00:00,2020-08-14 00:00:00,Closed,Completed,True


In [32]:
memory = ConversationBufferMemory(memory_key="chat_history")
dataset = {'df': dset}
tools = [PythonAstREPLTool(locals=dataset)]
tool_names = [tool.name for tool in tools]
prompt = OpenAIFunctionsAgent.create_prompt(system_message=SystemMessage(content='You are an intelligent chatbot capable of running Python code to do dataset exploration and machine learning. You are working with python pandas DataFrame with name `df`, read from the document text from the document loader for context. You should solve the question by small logical incremental steps and use the tools available.'))
agent = AgentExecutor.from_agent_and_tools(
    agent=OpenAIFunctionsAgent(llm=gchatllm, prompt=prompt, tools=tools, verbose=True),
    tools=tools, 
    verbose=True,
    memory=memory,
    handle_parsing_errors=True                          
)

In [33]:
agent.run("from the dataset determine which area path has the most work items in a new state.")



> Entering new AgentExecutor chain...
```python
import pandas as pd

# Get the unique area paths
area_paths = df['Area Path'].unique()

# Create a new DataFrame to store the results
results = pd.DataFrame(columns=['Area Path', 'Work Items in New State'])

# Iterate over the area paths
for area_path in area_paths:
    # Filter the DataFrame to only include rows for the current area path
    area_df = df[df['Area Path'] == area_path]

    # Count the number of work items in a new state
    new_state_count = area_df[area_df['State'] == 'New'].count()['Work Item ID']

    # Add the results to the new DataFrame
    results = results.append({'Area Path': area_path, 'Work Items in New State': new_state_count}, ignore_index=True)

# Sort the results by the number of work items in a new state
results = results.sort_values('Work Items in New State', ascending=False)

# Get the area path with the most work items in a new state
area_path_with_most_new_work_items = results['Area Path'].iloc[0]

#

"```python\nimport pandas as pd\n\n# Get the unique area paths\narea_paths = df['Area Path'].unique()\n\n# Create a new DataFrame to store the results\nresults = pd.DataFrame(columns=['Area Path', 'Work Items in New State'])\n\n# Iterate over the area paths\nfor area_path in area_paths:\n    # Filter the DataFrame to only include rows for the current area path\n    area_df = df[df['Area Path'] == area_path]\n\n    # Count the number of work items in a new state\n    new_state_count = area_df[area_df['State'] == 'New'].count()['Work Item ID']\n\n    # Add the results to the new DataFrame\n    results = results.append({'Area Path': area_path, 'Work Items in New State': new_state_count}, ignore_index=True)\n\n# Sort the results by the number of work items in a new state\nresults = results.sort_values('Work Items in New State', ascending=False)\n\n# Get the area path with the most work items in a new state\narea_path_with_most_new_work_items = results['Area Path'].iloc[0]\n\n# Print the re

In [11]:
agent.run("which area path has the most work items in a new state?")



> Entering new AgentExecutor chain...
```python
import pandas as pd

df = pd.read_csv("data/work_items.csv")

new_state_df = df[df["state"] == "New"]

area_path_counts = new_state_df["area_path"].value_counts()

print(area_path_counts.head())
```

> Finished chain.


'```python\nimport pandas as pd\n\ndf = pd.read_csv("data/work_items.csv")\n\nnew_state_df = df[df["state"] == "New"]\n\narea_path_counts = new_state_df["area_path"].value_counts()\n\nprint(area_path_counts.head())\n```'

Try create_pandas_dataframe_agent

In [ ]:
from langchain.agents import AgentType, create_pandas_dataframe_agent

In [ ]:
agent = create_pandas_dataframe_agent(
    llm, 
    dset, 
    verbose=True,
    return_intermediate_steps=True,
    max_iterations=3,
)                     


Put in Agent Executor

In [ ]:
agent = AgentExecutor.from_agent_and_tools(
    agent= create_pandas_dataframe_agent(llm, dset, verbose=True),
    tools=tools, 
    verbose=True,
    memory=memory,
    handle_parsing_errors=True                          
)